### EDA Report from Newcomer

Hi everyone, I'm new to Data Scientist. This is my work to find insights of a strange dataset. This is not a finished version. Thank for reading

### 1. Loading dataset

In [ ]:
# Loading libraries for processing data
import numpy as np
import pandas as pd

# Loading visualization libraries
import seaborn as sns
import matplotlib.pyplot as plt

# Formating the code
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# reading data

items = pd.read_csv('../input/competitive-data-science-predict-future-sales/items.csv')
items_cate = pd.read_csv('../input/competitive-data-science-predict-future-sales/item_categories.csv')
sale = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv')
shop = pd.read_csv('../input/competitive-data-science-predict-future-sales/shops.csv')
test = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')

In [ ]:
# take a quick look at the data

items.head()

In [ ]:
items.describe().T

In [ ]:
items_cate.head()

In [ ]:
items_cate.describe().T

In [ ]:
sale.head()

In [ ]:
sale.describe().T

In [ ]:
shop.head()

In [ ]:
shop.describe().T

In [ ]:
test.head()

In [ ]:
test.describe().T

### 2. Understanding the data

*Data fields*

ID - an Id that represents a (Shop, Item) tuple within the test set

shop_id - unique identifier of a shop

item_id - unique identifier of a product

item_category_id - unique identifier of item category

item_cnt_day - number of products sold. You are predicting a monthly amount of this measure

item_price - current price of an item

date - date in format dd/mm/yyyy

date_block_num - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33

item_name - name of item

shop_name - name of shop

item_category_name - name of item category

#### 2.1 Data hypothesis

- The purpose of this work is to predict monthly amount of item_cnt_day. After having the quick view of the data, there are several ways that we can deepdive into this dataset:
       1. Which item categories are selling better than others?
       2. Are there seasons for selling by looking at total sale/month?
       3. Which shops are doing good, which are doing badly and in order to increase revenue, should we invest more into profitable ones and shut-down the unprofitable ones?
       4. Are items price effect the amount of sale and why so?
       5. Which items are selling better than others? Is the best-seller items changed based on shop or seasons?

#### 2.3 Quick visualization for data understanding

In [ ]:
# no of items per cate

x = items.groupby(['item_category_id']).count()
x = x.sort_values(by='item_id', ascending=False)
x = x.reset_index()
x.head()

In [ ]:
# no of sale by shop
y = sale.groupby(['shop_id']).sum()
y = y.reset_index()
y.head()

In [ ]:
# no of sale by price range
z = sale.sort_values('item_price')
z = z.groupby(pd.cut(z['item_price'], np.arange(z['item_price'].min(), z['item_price'].max(), z['item_price'].max()/200))).sum()

# bins = np.arange(z['item_price'].min(), z['item_price'].max(), z['item_price'].max()/100)
# z = z.groupby('item_price').sum()
# z = z.reset_index()
# z.describe()

In [ ]:
z = z.rename({'item_price': 'price'}, axis=1)
z = z.iloc[0:20].reset_index()
z = z.rename({'item_price': 'price_range'}, axis=1)
z.head()

In [ ]:
z = z.reset_index()
z

In [ ]:
g = sale.groupby(['date_block_num']).sum()
g = g.reset_index()
g.head()

In [ ]:
#visualization 

f, axes = plt.subplots(2,2, figsize=(25,15))

sns.barplot(x='item_category_id', y='item_id', data=x, ax=axes[0,0]).set_title('Cate amount')
sns.barplot(x='shop_id', y='item_cnt_day', data=y, ax=axes[0,1]).set_title('Shop sale')
sns.barplot(x='index', y='item_cnt_day', data=z, ax=axes[1,0]).set_title('Price range sale')
sns.barplot(x='date_block_num', y='item_cnt_day', data=g, ax=axes[1,1]).set_title('Monthly sale')

By quick look, I assump that price range has outliers, therefore I will remove outliers so that I can have a better view of price range

In [ ]:
plt.figure(figsize=(20,10))
sns.jointplot(x='item_cnt_day', y='item_price', data=sale)

In [ ]:
a = sale['item_price'].nlargest(50)
a = a.iloc[49,]
a

In [ ]:
h = sale.groupby(pd.cut(sale['item_price'], np.arange(sale['item_price'].min(), a, a/300))).sum()
h['price_range']=np.arange(len(h))
h = h.iloc[0:20]

In [ ]:
plt.figure(figsize=(20,10))
sns.barplot(h.price_range, h.item_cnt_day)

In order to map the details of the data, I will join small table into a full table.

In [ ]:
# Join data

full = sale.join(items, on='item_id', rsuffix='_').join(shop, on='shop_id', rsuffix='_').join(items_cate, on='item_category_id', rsuffix='_').drop(['item_id_', 'shop_id_', 'item_category_id_'], axis=1)
full.head()

In [ ]:
full.describe()

In [ ]:
full['revenue'] = full['item_cnt_day'] * full['item_price']
full.head()

In [ ]:
# full['revenue'] =full['item_price'].apply(lambda x: (x * full['item_cnt_day']))

In [ ]:
revenue_month = full.groupby(['date_block_num'], as_index=False).sum()
revenue_month.head()

In [ ]:
plt.figure(figsize=(20,10))
sns.barplot(x=full.date_block_num, y=full.revenue).set_title('Monthly Revenue')

In [ ]:
# change time format to do comparison 
full['year'] = full['date_block_num'].apply(lambda x: ((x//12) + 2013))
full['month'] = full['date_block_num'].apply(lambda x: (x %12) +1 )
full.head()

In [ ]:
full.tail()

In [ ]:
month = full[['month', 'year', 'item_cnt_day']]
month = month.sort_values(['month'])
month

In [ ]:
month_pivot = month.pivot_table(index='month', columns='year', values='item_cnt_day')
month_pivot

In [ ]:
full

In [ ]:
revenue_month = full[['month', 'year', 'revenue']]
revenue_pivot = revenue_month.pivot_table(index='month', columns='year', values='revenue')
revenue_pivot

In [ ]:
f, axes = plt.subplots(2, 1, figsize=(22,10))
month_pivot.plot(kind='bar', rot=0, ax=axes[0]).set_title('Monthly sale')                                       
revenue_pivot.plot(kind='bar', rot=0, ax=axes[1]).set_title('Monthly revenue') 